In [108]:
import requests
from bs4 import BeautifulSoup
from datetime import date , timedelta
from tqdm import tqdm
import time
import pandas as pd

#NewsBiscuit - United Kingdom:  http://www.newsbiscuit.com/

Trang báo NewsBiscuit cho phép xem các báo phát hành theo ngày (ví dụ với đường dẫn http://www.newsbiscuit.com//2021/06/05 ta có thể xem tất cả các bài báo đã phát hành vào ngày 05/06/2021)nên ta có thể dễ dàng crawl tiêu đề bài báo ở một ngày cụ thể với url dạng http://www.newsbiscuit.com//yyyy/mm/dd)

In [ ]:
today  = date.today()
d1 = today
d2 = d1.strftime("%Y/%m/%d") # lấy thời điểm ngày hiện tại
print(d2)

2021/06/10


In [ ]:
link  = "http://www.newsbiscuit.com/" # đường dẫn trang web

In [ ]:
def get_data(link,d2):
  lst = []
  link = link + d2 + "/" # lấy tiêu đề trang web theo từng ngày  
  response = requests.get(link) # 
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.findAll(class_ = "col-sm-8 text-center") # tìm tất cả các thẻ chưa thông tin bài báo
  links = [link.find('a').attrs["href"] for link in table] # lấy tất cả các đường link  của các bài báo
  headline = [link.find('a').get_text() for link in table] # lấy tất cả các headline của các bài báo
  for i in range(0,len(links)): # ghép đường link và headline phù hợp với nhau 
    dic = dict()
    dic["article_link"] = links[i]
    dic["headline"] = headline[i]
    dic["is_sarcastic"] = 1
    lst.append(dic)
  return lst

In [ ]:
lst =[]
while d2 != "2018/12/31": # lấy trong vòng 3 năm t
  d1 = d1 - timedelta(days=1)
  d2 = d1.strftime("%Y/%m/%d")
  lst += get_data(link, d2) 
  

In [44]:
#lưu thành file csv
headline = []
article_link = []
is_sarcastic = []
for i in lst:
  article_link.append(i['article_link'])
  headline.append(i['headline'])
  is_sarcastic.append(i['is_sarcastic'])


In [48]:
df = pd.DataFrame({'headline':headline,'article_link':article_link,'is_sarcastic':is_sarcastic})

In [49]:
df.head()

,headline,article_link,is_sarcastic
0,Southgate selects 26 goalkeepers,http://www.newsbiscuit.com/2021/06/07/southgat...,1
1,Hospitality Crisis: BBC to Show “Great British...,http://www.newsbiscuit.com/2021/06/06/hospital...,1
2,China says ‘nothing to see here’ on Tiananmen ...,http://www.newsbiscuit.com/2021/06/06/china-sa...,1
3,Top Gear presenters to pick up remaining audie...,http://www.newsbiscuit.com/2021/06/06/top-gear...,1
4,British holidaymakers jubilant as budget desti...,http://www.newsbiscuit.com/2021/06/06/british-...,1


In [50]:
df.to_csv('NewsBiscuit.csv')

#The True North Times - Canada: http://www.truenorthtimes.ca/

Trang báo True North Times cho phép xem các bài báo đã đăng theo từng danh mục nên ta sẽ duyệt qua các danh mục để lấy tiêu đề và đường dẫn các bài báo đã đăng

In [ ]:
link2 = "http://www.truenorthtimes.ca/category/" # lấy đường dẫn và danh sách category
list_category = [{'name' : "federal-politics", 'page': 47}, {'name':"regional-politics", 'page': 35} , {'name': "foreign-affairs" ,'page': 6}, {'name':"columns", 'page': 9}, {'name': "interactive", 'page':5}, {'name':"interviews", 'page':2} ]

In [ ]:
def get_data2(link): #hàm lấy tiêu đề và đường dẫn trong bài báo
  lst = []
  response = requests.get(link)
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.findAll("h2", class_="cb-post-title")
  dic =dict ()
  links = [link.find('a').attrs["href"] for link in table]
  headline = [link.find('a').text for link in table]
  for i in range(1,len(links)):
    dic = dict()
    dic["article_link"] = links[i]
    dic["headline"] = headline[i]
    dic["is_sarcastic"] = 1
    lst.append(dic)
  return lst[4:] # do 4 bài báo đầu sẽ bị trùng lập lên không lấy
  

In [63]:
print(len(lst)) # do trang báo được thành lập bởi một nhóm sinh viên vào năm 2014 nhưng bài báo đăng cuối cùng vào đầu năm 2017 nên số lượng thu được ít

1008


In [ ]:
lst = []
i = 0     
for i in range(len(list_category)): #duyệt qua các thư mục để lấy các bài báo trong các thư mục
  for j in range(1,list_category[i]['page']+1):
      detail_link = link2 + list_category[i]['name'] + "/page/"+str(j) + "/" #chỉnh sửa lại đường dẫn
      print(detail_link)
      lst += get_data2(detail_link) # lấy danh sách các bài báo
      

In [64]:
def extra_csv(lst):
  headline = []
  article_link = []
  is_sarcastic = []
  for i in lst:
    article_link.append(i['article_link'])
    headline.append(i['headline'])
    is_sarcastic.append(i['is_sarcastic'])
  df = pd.DataFrame({'headline':headline,'article_link':article_link,'is_sarcastic':is_sarcastic})
  return df
df =extra_csv(lst)

In [66]:
df.head()

,headline,article_link,is_sarcastic
0,Nation Reeling After Prime Minister Trudeau Fo...,http://www.truenorthtimes.ca/2017/07/02/trudea...,1
1,Kevin O’Leary Quits Conservative Leadership Ra...,http://www.truenorthtimes.ca/2017/04/26/kevin-...,1
2,Globe & Mail Columnist Claims She Breastfed Mi...,http://www.truenorthtimes.ca/2017/03/27/globe-...,1
3,(Interview) Will Michael Chong Make Canada Gre...,http://www.truenorthtimes.ca/2017/03/22/interv...,1
4,Kellie Leitch Lets Her Canadian Values Hang Out,http://www.truenorthtimes.ca/2017/02/02/kellie...,1


In [65]:
df.to_csv('TheTrueNorthTime.csv') # lưu thành file csv

#The Betoota Advocate - Australia: http://betootaadvocate.com/

Trang báo Betoota Advocate cho phép xem các bài báo đã đăng theo từng danh mục nên ta sẽ duyệt qua các danh mục để lấy tiêu đề và đường dẫn các bài báo đã đăng

In [ ]:
link3 = "https://www.betootaadvocate.com/category/" # Đường dẫn đến các thư mục của trang
list_category = [{'name' : "breaking-news", 'page': 144}, {'name':"advocate-in-focus", 'page': 90} , {'name': "entertainment" ,'page': 75}, {'name':"sports", 'page': 105}, {'name': "uncategorized", 'page':450}, {'name':"humans-of-betoota", 'page':160}, {'name':"world-news", 'page':55} ]

In [ ]:
def get_data3(link):
  lst = []
  response = requests.get(link)
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.findAll("h3", class_="entry-title td-module-title")
  dic =dict ()
  links = [link.find('a').attrs["href"] for link in table]
  headline = [link.find('a').text for link in table]
  for i in range(1,len(links)):
    dic = dict()
    dic["article_link"] = links[i]
    dic["headline"] = headline[i]
    dic["is_sarcastic"] = 1
    lst.append(dic)
  return lst[2:12]

In [ ]:
from tqdm import tqdm
lst = []
i = 0
for i in tqdm(range(len(list_category))):
  for j in range(1,list_category[i]['page']+1):
      detail_link = link3 + list_category[i]['name'] + "/page/"+str(j) + "/"
      lst += get_data3(detail_link)

100%|██████████| 7/7 [46:26<00:00, 398.03s/it]


In [85]:
print(len(lst))

10782


In [86]:
df = extra_csv(lst)

In [87]:
df.head()

,headline,article_link,is_sarcastic
0,Millennial Can’t Afford Home After Spending Al...,https://www.betootaadvocate.com/breaking-news/...,1
1,Oh No: Best Man’s Speech Got Enough Laughs For...,https://www.betootaadvocate.com/breaking-news/...,1
2,Regional Dad Offers Monosyllabic Words Of Appr...,https://www.betootaadvocate.com/breaking-news/...,1
3,"“Yeah I’ve Got The AstraZeneca, Just Leaving M...",https://www.betootaadvocate.com/breaking-news/...,1
4,QLD Government Purchases 100 Old Nokia Batteri...,https://www.betootaadvocate.com/breaking-news/...,1


In [88]:
df.to_csv('Betootaadcocate.csv') # Lưu thành file csv

#The NewYork Time 
https://www.nytimes.com/


Trang báo The New York Time cho phép xem các báo phát hành theo ngày (ví dụ với đường dẫn http://www.newsbiscuit.com//2021/06/05/todays-new-york-times ta có thể xem tất cả các bài báo đã phát hành vào ngày 05/06/2021) nên ta có thể dễ dàng crawl tiêu đề bài báo ở một ngày cụ thể với url dạng http://www.nytimes.com//yyyy/mm/dd/todays-new-york-times)

In [ ]:
def get_data4(link): 
  lst = []
  response = requests.get(link)
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.findAll("div", class_="css-141drxa")
  dic =dict ()
  links = [link.find('a').attrs["href"] for link in table] #Tìm kiếm đường link
  headline = [link.find('a').text for link in table] #tìm kiếm tiêu đề bài báo
  for i in range(0,len(links)): #ghép các nội dung lại với nhau
    dic = dict()
    dic["article_link"] = "https://www.nytimes.com"+ links[i]
    dic["headline"] = headline[i]
    dic["is_sarcastic"] = 0
    lst.append(dic)
  return lst

In [ ]:
today  = date.today() #lấy ngày hiện tại
d1 = today
d2 = d1.strftime("%Y/%m/%d")
print(d2)

2021/06/08


In [ ]:
link4 ="https://www.nytimes.com/issue/todayspaper/" 

In [ ]:
lst =[]
while d2 != "2017/12/31":  
  d2 = d1.strftime("%Y/%m/%d")
  d1 = d1 - timedelta(days=1)
  link4_ = link4 + d2+  "/todays-new-york-times" #nối đường link bài báo đến ngày cần tìm
  lst += get_data4(link4_) #lấy danh sách các bài báo 
  
  

In [97]:
len(lst)

53818

In [99]:
df = extra_csv(lst) 

In [101]:
df
  

,headline,article_link,is_sarcastic
0,"2 Trains Collide in Pakistan, Killing at Least...",https://www.nytimes.com/2021/06/07/world/asia/...,0
1,"In Arizona 2020 Election Review, Risks for Rep...",https://www.nytimes.com/2021/06/07/us/arizona-...,0
2,"A House Divided: A Palestinian, a Settler and ...",https://www.nytimes.com/2021/06/07/world/middl...,0
3,"In Guatemala, Harris Tells Undocumented to Sta...",https://www.nytimes.com/2021/06/07/world/ameri...,0
4,Rome Gets Its First Pizza Vending Machine. Wil...,https://www.nytimes.com/2021/06/07/world/europ...,0
...,...,...,...
53813,"Kelly Jones, Daniel Howell",https://www.nytimes.com/2017/12/31/fashion/wed...,0
53814,"Manhattan Prices Stable in 2017, Even as Luxur...",https://www.nytimes.com/2017/12/29/realestate/...,0
53815,Landlords and Sellers AdjustBy Michael Kolomatsky,https://www.nytimes.com/2017/12/28/realestate/...,0
53816,Can I Divide My Apartment Into Two?By Ronda Ka...,https://www.nytimes.com/2017/12/30/realestate/...,0


In [102]:
df.to_csv("TheNewYorkTimes.csv") #lưu file

# https://nationalpost.com/


Trang báo National Post cho phép xem các bài báo theo nội dung tìm kiếm 
- Mỗi lần tìm kiếm sẽ được một list 10 bài báo và ta sẽ duyệt qua các list bài báo tìm kiếm được để thu thập các tiêu đề và đường link bài báo

In [103]:
link5 = "https://nationalpost.com/search/?search_text=news&date_range=-7300d&sort=desc&from="

In [106]:
def get_data5(link5):
  response = requests.get(link5)
  lst = []
  soup = BeautifulSoup(response.content, "html.parser")
  table  = soup.findAll('div', class_="article-card__content") #Tìm list bài báo
  times = []
  for i in table: #tìm thẻ span chứa ngày tháng năm phát hành bài báo
    time = []
    z = i.find('div') 
    for j in z:
      k = j.find('span')
      time.append(k)
    year = time[3].text[-5:]
    try :
      if (year == ' ago '):
        times.append(2021)
      else:
        times.append(int(year))
    except:
      times.append(2018)
  links = [link.find('a').attrs["href"] for link in table] #tìm các đường link bài báo
  headline = []    #find headline
  for link in table: #Tìm tiêu đề bài báo
    text = link.find('a').text.split('   ') 
    headline.append(text[0])
  for i in range(len(links)):
    dic = dict()
    dic["article_link"] = "https://nationalpost.com/"+ links[i]
    dic["headline"] = headline[i]
    dic["is_sarcastic"] = 0
    print(dic)
    if (int(times[i])> 2018): #thêm nhưng bài báo phát hành sau năm 2018
      lst.append(dic)
  return lst  

In [ ]:
lst = []
for index in tqdm(range(0, 10000,10)): #tìm kiếm 10000 bài báo
  link = link5 + str(index)
  lst += get_data5(link)

In [117]:
df =extra_csv(lst) 

In [118]:
df.to_csv("NationalPost.csv") #Lưu file csv

# Crawl data from Euronews

Trang báo Euronews cho phép xem các báo phát hành theo ngày (ví dụ với đường dẫn https://www.euronews.com/2021/06/05 ta có thể xem tất cả các bài báo đã phát hành vào ngày 05/06/2021)nên ta có thể dễ dàng crawl tiêu đề bài báo ở một ngày cụ thể với url dạng https://www.euronews.com/yyyy/mm/dd)

In [120]:
from datetime import date,timedelta
today = date.today()
d = today.strftime("%Y/%m/%d")
d

'2021/06/10'

In [121]:
headline = []
article_link = []
is_sarcastic = []
date = []
link = 'https://www.euronews.com/' # Đường dẫn đến trang web
while(today.year>=2019):
  req = requests.get(link+d) # d là yyyy/mm/dd
  if(req.status_code==200): # Kiểm tra request thành công hay không?
    soup = BeautifulSoup(req.text, "lxml")

    for i in soup.find_all('a',class_ = 'm-object__title__link '):
      headline.append(i.get('title')) #Lấy tiêu đề 
      article_link.append('https://www.euronews.com/'+i.get('href')) #Lấy URL
      is_sarcastic.append(0)
      date.append(d)
  
  today -= timedelta(1)
  d = today.strftime("%Y/%m/%d")


In [122]:
df = pd.DataFrame({'headline':headline,'article_link':article_link,'is_sarcastic':is_sarcastic,'date':date})
df

,headline,article_link,is_sarcastic,date
0,Ranked: The top 10 most liveable cities in the...,https://www.euronews.com//travel/2021/06/10/ra...,0,2021/06/10
1,Two friends start a renewable revolution on th...,https://www.euronews.com//green/2021/06/10/two...,0,2021/06/10
2,"Lisbon expresses ""regret"" after sharing detail...",https://www.euronews.com//2021/06/10/lisbon-ex...,0,2021/06/10
3,A classic Belgian Waffle but with a crunchy tw...,https://www.euronews.com//green/2021/06/10/bel...,0,2021/06/10
4,Campaigners demand G7 leaders give firm commit...,https://www.euronews.com//2021/06/10/campaigne...,0,2021/06/10
...,...,...,...,...
23991,Parched Jordan to drill a half-mile undergroun...,https://www.euronews.com//2019/01/01/jordan-dr...,0,2019/01/01
23992,"In 2019, the nuclear deal is hanging by a thre...",https://www.euronews.com//2019/01/01/the-iran-...,0,2019/01/01
23993,Feeling lonely? You're not alone. How to find ...,https://www.euronews.com//2019/01/01/lonely-yo...,0,2019/01/01
23994,France: Eight people spend New Year's Eve stuc...,https://www.euronews.com//2019/01/01/france-ei...,0,2019/01/01


In [123]:
df.to_csv('euronews.csv')